<a href="https://colab.research.google.com/github/annashorikova/cl_python/blob/main/Hw2_%D0%A8%D0%BE%D1%80%D0%B8%D0%BA%D0%BE%D0%B2%D0%B0_%D0%90%D0%BD%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 2

## pandas + парсинг

Здесь находится главная страница [Системного блока](https://sysblok.ru)

Соберите корпус новостей (можно спарсить все новости или небольшую часть)

Дедлайн: 31 декабря включительно

In [ ]:
# необходимые импорты
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

### шаг 1

In [ ]:
# соберите все ссылки на страницы с новостями (цикл for + range)
# 1 страница: https://sysblok.ru
# 2 страница: https://sysblok.ru/page/2
# последняя: https://sysblok.ru/page/16

all_page = ['https://sysblok.ru']
for i in range(2, 17):
  all_page.append('https://sysblok.ru/page/' + str(i))
print(all_page)

### шаг 2

In [94]:
# с каждой страницы соберите ссылки на отдельные новости
# подсказка: родительский тег h2 (проверяйте по нему), нужный нам - a

all_links = []
for i in all_page:
  page = rq.get(i)
  page.encoding = 'utf-8'
  soup = BeautifulSoup(page.text, features="html.parser")
  for i in soup.find_all('a'):
    if i.parent.name == 'h2':
      all_links.append(i.get('href'))

### шаг 3

работаем с 1 новостью:

нас интересует название, автор, дата публикации, текст (дополнительно можно спарсить тематические категории)

In [ ]:
# для парсинга даты: извлеките текст из тега time

print(BeautifulSoup(rq.get(all_links[0]).text, features="html.parser").find('time').text)

29.12.2023


In [ ]:
# для парсинга заголовка: извлеките текст из тега h1

print(BeautifulSoup(rq.get(all_links[0]).text, features="html.parser").find('h1').text)

Что читают на программе по Digital Humanities? Выбор преподавателей DH-центра ИТМО


In [ ]:
# для парсинга текста: соберите все тексты из тега p и соедините в строку

article_text = ''
for i in BeautifulSoup(rq.get(all_links[0]).text, features="html.parser").find_all('p'):
    article_text += i.text.strip()
print(article_text)

In [ ]:
# автора собрать сложнее:
# достаем из тега meta как на паре или из тега a с проверкой на содержимое атрибута rel

print(BeautifulSoup(rq.get(all_links[0]).text, features="html.parser").find_all('meta', {'name' : 'author'})[0].attrs['content'].split(','))

['DH Center ITMO University', ' Вика Книппер']


In [ ]:
# тематические категории:
# <a <a rel= tag"> нужные нам категории </a>
# можно попробовать так:
# if i.get('rel') == ['category', 'tag']

category = []
for i in BeautifulSoup(rq.get(all_links[0]).text, features="html.parser").find_all('a'):
  if i.get('rel') == ['category', 'tag']:
    category.append(i.text)


['Блог DH ИТМО', 'Блоги']


### шаг 4

In [95]:
# пишем функцию по сбору информации
# функция собирает всю информацию о статье, которую мы находили ранее, в список

def article_data(article):
  # в этот список будем собирать все данные по статье
  data = []

  # добавляем ссылку на статью
  data.append(article)

  # добавляем дату
  data.append(BeautifulSoup(rq.get(article).text, features="html.parser").find('time').text)

  # добавляем заголовок
  data.append(BeautifulSoup(rq.get(article).text, features="html.parser").find('h1').text)

  # добавляем список авторов
  data.append(BeautifulSoup(rq.get(article).text, features="html.parser").find_all('meta', {'name' : 'author'})[0].attrs['content'].split(','))

  # добавляем список категорий
  category = []
  for i in BeautifulSoup(rq.get(article).text, features="html.parser").find_all('a'):
    if i.get('rel') == ['category', 'tag']:
      category.append(str(i.text))
  data.append(category)

  # добавляем текст статьи
  article_text = ''
  for i in BeautifulSoup(rq.get(article).text, features="html.parser").find_all('p'):
      article_text += i.text.strip()
  data.append(article_text)

  return data

In [ ]:
# тестируем на одной статье

article_data(all_links[0])

In [96]:
# собираем все новости!
# полезно использовать try / except

dataset = [] # в этот список будем собирать списки с данными по каждой статье
for i in range(100):
  try:
    dataset.append(article_data(all_links[i]))
  except:
    print(f'Не работает со статьей {all_links[i]}')

### шаг 5

In [ ]:
# соберите всю собранную информацию в датафрейм

df = pd.DataFrame(dataset, columns=['URL', 'Date', 'Title', 'Authors', 'Categories', 'Text'])

df

In [98]:
df.to_csv("sysblock_articles.csv", encoding='utf-8')

Готово!
Загрузите ваш код и собранные новости на GitHub